# Homework Reflections – Weeks 9, 11, and 12

This notebook collects all code used for the reflections:
- Week 9: Heteroskedasticity, correlated errors, and bootstrap vs full DGP
- Week 11: Event study dataset and model
- Week 12: Difference-in-differences with violated prior trends


## Week 9 – Simulation Reflections

### Week 9 – Question 1
**Task:** Write code that uses a simulation to estimate the standard deviation of the coefficient when there is heteroskedasticity, and compare these standard errors to those from OLS.

In [ ]:
import numpy as np
import statsmodels.api as sm

rng = np.random.default_rng(0)

def sim_hetero_once(n=1000, beta=1.0, sigma=1.0, h=1.5):
    """DGP: Y = beta*X + e, with heteroskedastic errors: Var(e|X) = sigma^2*(1 + h*|X|)^2"""
    X = rng.normal(0, 1, n)
    e = rng.normal(0, sigma * (1 + h * np.abs(X)))
    Y = beta * X + e
    X1 = sm.add_constant(X)
    model = sm.OLS(Y, X1).fit()
    # HC1 robust
    model_hc1 = model.get_robustcov_results(cov_type="HC1")
    return {
        "beta_hat": model.params[1],
        "se_conventional": model.bse[1],
        "se_hc1": model_hc1.bse[1]
    }

def sim_hetero_many(R=1000, n=1000, beta=1.0, sigma=1.0, h=1.5):
    out = [sim_hetero_once(n=n, beta=beta, sigma=sigma, h=h) for _ in range(R)]
    bhats = np.array([d["beta_hat"] for d in out])
    se_conv = np.array([d["se_conventional"] for d in out])
    se_hc1  = np.array([d["se_hc1"] for d in out])
    return {
        "emp_sd_beta_hat": bhats.std(ddof=1),
        "mean_se_conventional": se_conv.mean(),
        "mean_se_hc1": se_hc1.mean()
    }

res_het = sim_hetero_many(R=1000, n=1000, beta=1.0, sigma=1.0, h=1.5)
res_het

### Week 9 – Question 2
**Task:** Use a simulation to estimate the standard deviation of the coefficient when errors are highly correlated / non-independent, and compare these standard errors to those from OLS.

In [ ]:
import numpy as np
import statsmodels.api as sm

rng = np.random.default_rng(1)

def sim_cluster_once(G=100, m=10, beta=1.0, tau=1.2, sigma=1.0):
    """DGP: G clusters, m obs per cluster (n = G*m).
    X ~ N(0,1); errors have cluster shock u_g and idiosyncratic v.
    Y = beta*X + e,  e = u_g + v
    """
    n = G * m
    g_id = np.repeat(np.arange(G), m)
    X = rng.normal(0, 1, n)
    u = rng.normal(0, tau, G)  # cluster shocks
    v = rng.normal(0, sigma, n)
    e = u[g_id] + v
    Y = beta * X + e
    X1 = sm.add_constant(X)
    model = sm.OLS(Y, X1).fit()
    # Cluster-robust by group id
    model_clu = model.get_robustcov_results(cov_type="cluster", groups=g_id)
    return {
        "beta_hat": model.params[1],
        "se_conventional": model.bse[1],
        "se_cluster": model_clu.bse[1]
    }

def sim_cluster_many(R=1000, G=100, m=10, beta=1.0, tau=1.2, sigma=1.0):
    out = [sim_cluster_once(G=G, m=m, beta=beta, tau=tau, sigma=sigma) for _ in range(R)]
    bhats = np.array([d["beta_hat"] for d in out])
    se_conv = np.array([d["se_conventional"] for d in out])
    se_clu  = np.array([d["se_cluster"] for d in out])
    return {
        "emp_sd_beta_hat": bhats.std(ddof=1),
        "mean_se_conventional": se_conv.mean(),
        "mean_se_cluster": se_clu.mean()
    }

res_cluster = sim_cluster_many(R=1000, G=100, m=10, beta=1.0, tau=1.2, sigma=1.0)
res_cluster

### Week 9 – Question 3
**Task:** Show that with strong correlation, the bootstrap standard deviation of the coefficient (using naive iid residual bootstrap) can differ from the standard deviation from a full simulation of the DGP; show how this improves with a larger sample and a better bootstrap.

In [ ]:
import numpy as np, statsmodels.api as sm

rng = np.random.default_rng(0)

def dgp_cluster(G=40, m=8, beta=1.0, tau=3.0, sigma=1.0):
    """Strong intra-cluster correlation: rho = tau^2 / (tau^2 + sigma^2) ≈ 0.9."""
    n = G*m
    g = np.repeat(np.arange(G), m)
    X = rng.normal(0, 1, n)
    u = rng.normal(0, tau, G)         # cluster shock
    v = rng.normal(0, sigma, n)       # idiosyncratic
    e = u[g] + v
    Y = beta*X + e
    return Y, X, g

def iid_resid_boot_se(Y, X, B=800):
    X1 = sm.add_constant(X)
    base = sm.OLS(Y, X1).fit()
    resid = base.resid
    betas = []
    for _ in range(B):
        e_star = rng.choice(resid, size=len(resid), replace=True)  # WRONG under dependence
        Y_star = base.fittedvalues + e_star
        betas.append(sm.OLS(Y_star, X1).fit().params[1])
    return np.std(betas, ddof=1)

def cluster_boot_se(Y, X, groups, B=800):
    # Resample entire clusters (block bootstrap)
    X1 = sm.add_constant(X)
    base = sm.OLS(Y, X1).fit()
    betas = []
    gids = np.unique(groups)
    idx_by_g = {g: np.where(groups==g)[0] for g in gids}
    for _ in range(B):
        sample_g = rng.choice(gids, size=len(gids), replace=True)
        idx = np.concatenate([idx_by_g[g] for g in sample_g])
        betas.append(sm.OLS(Y[idx], X1[idx]).fit().params[1])
    return np.std(betas, ddof=1)

def empirical_sd_over_dgp(R=800, **dgp_kwargs):
    bhats = []
    for _ in range(R):
        Y, X, g = dgp_cluster(**dgp_kwargs)
        bhats.append(sm.OLS(Y, sm.add_constant(X)).fit().params[1])
    return np.std(bhats, ddof=1)

G, m = 40, 8
Y, X, g = dgp_cluster(G=G, m=m)
emp_sd = empirical_sd_over_dgp(R=800, G=G, m=m)
se_iid  = iid_resid_boot_se(Y, X, B=800)
se_clust= cluster_boot_se(Y, X, g, B=800)

small_sample_results = {
    "empirical_SD_DGP": emp_sd,
    "SE_naive_IID_boot": se_iid,
    "SE_cluster_boot": se_clust
}
small_sample_results

In [ ]:
# Larger sample: more clusters and/or size per cluster
G, m = 400, 20
Y_big, X_big, g_big = dgp_cluster(G=G, m=m)
emp_sd_big = empirical_sd_over_dgp(R=400, G=G, m=m)
se_iid_big  = iid_resid_boot_se(Y_big, X_big, B=400)
se_clust_big= cluster_boot_se(Y_big, X_big, g_big, B=400)

large_sample_results = {
    "empirical_SD_DGP_big": emp_sd_big,
    "SE_naive_IID_boot_big": se_iid_big,
    "SE_cluster_boot_big": se_clust_big
}
large_sample_results

## Week 11 – Event Study Reflection

### Week 11 – Question 1(a)
**Task:** Construct a dataset for an event study where the value, derivative, and second derivative of a trend all change discontinuously after an event, and build a model using only the value.

In [ ]:
import numpy as np
import statsmodels.api as sm

np.random.seed(0)

# Set up time and event
T = 200
t = np.arange(T)
event_time = 100
post = (t > event_time).astype(int)

# Construct Y with different level, slope, and curvature after the event
# Pre-event: mild quadratic
# Post-event: bigger intercept, steeper slope, more curvature
Y = np.where(
    t <= event_time,
    0.01 * t**2 + 0.1 * t,                       # pre-event trend
    (0.03 * t**2 + 0.6 * t + 10),                # post-event trend: level, slope, curvature all changed
)

# Add noise
Y = Y + np.random.normal(0, 1, size=T)

# Model using ONLY the value (no explicit derivatives),
# but allowing for a general trend via t and t^2
X = np.column_stack([post, t, t**2])   # event indicator + time + time^2
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
print(model.summary())

# The key question: is the event "real"?
# That is, is the coefficient on `post` (the event dummy) significantly nonzero?
beta_event = model.params[1]
pval_event = model.pvalues[1]
print("Event coefficient (beta_event):", beta_event)
print("p-value for event effect:", pval_event)

## Week 12 – Difference-in-Differences with Violated Prior Trends

### Week 12 – Prior Trends Do Not Hold
**Task:** Construct a dataset in which prior trends do not hold, and in which this makes the differences-in-differences estimate come out wrong.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(0)

# Number of observations per (group, time) cell
n_per_cell = 100

# Time periods and groups
times = np.arange(-5, 5)        # -5, -4, ..., 4
groups = np.array([0, 1])       # 0 = control, 1 = treatment

# Create all (time, group) combinations and repeat within each cell
time_grid, group_grid = np.meshgrid(times, groups, indexing='ij')
time_vec = np.repeat(time_grid.ravel(), n_per_cell)
group_vec = np.repeat(group_grid.ravel(), n_per_cell)

# Indicator for post-treatment period
post = (time_vec >= 0).astype(int)

# Different pre-trends: treatment group has a steeper slope
control_trend = 0.5 * time_vec
treat_trend   = 1.0 * time_vec   # steeper line

trend = np.where(group_vec == 0, control_trend, treat_trend)

# True treatment effect
true_effect = 2.0

# Outcome with noise
noise = np.random.normal(0, 1, len(time_vec))
Y = trend + true_effect * post * group_vec + noise

# Put into a DataFrame
df = pd.DataFrame({
    "Y": Y,
    "Time": time_vec,
    "Group": group_vec,
    "Post": post
})

# DID regression: Y ~ Group + Post + Group*Post
df["interaction"] = df["Group"] * df["Post"]
X = sm.add_constant(df[["Group", "Post", "interaction"]])
res = sm.OLS(df["Y"], X).fit()

print(res.summary())
print("True treatment effect:", true_effect)
print("Estimated DID effect:", res.params["interaction"])